In [1]:
import logging

import probtorch
import torch

import combinators
import smc
import hmm
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float).unsqueeze(0) * 2,
        'scale': torch.ones(1, 5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(1, 5),
        'scale': torch.ones(1, 5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(1, 5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, theta=hmm_params)
init_hmm.condition(generative)

In [5]:
z0, mu, sigma, pi = init_hmm(T=50)

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.partial(combinators.sequence, hmm_step)

In [7]:
hmm_run.condition(generative)

In [8]:
z_last, mu, sigma, pi = hmm_run(50, z0, mu, sigma, pi)

In [9]:
generative = generative.squeeze()

In [10]:
num_particles = 100

In [11]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.zeros(5),
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    smc_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [12]:
smc_hmm = smc.smc(hmm.hmm_step, hmm.hmm_retrace, reparameterized=False)

In [13]:
inference_init = combinators.Model(hmm.init_hmm, smc_hmm_params, {})

inference = smc.ParticleTrace(num_particles)
inference_init.condition(trace=inference, observations=generative)
smc_hmm.condition(trace=inference, observations=generative)

In [14]:
inference, smc_hmm_params = smc.variational_smc(num_particles, hmm.init_hmm, smc_hmm, 500, 50, smc_hmm_params, generative, use_cuda=False, marginal_model='hmm_step', lr=1e-2)

07/12/2018 13:35:09 Variational SMC ELBO=-1.81135025e+02 at epoch 1
07/12/2018 13:35:09 Variational SMC ELBO=-1.70687500e+02 at epoch 2
07/12/2018 13:35:10 Variational SMC ELBO=-1.89331543e+02 at epoch 3
07/12/2018 13:35:10 Variational SMC ELBO=-1.65243958e+02 at epoch 4
07/12/2018 13:35:10 Variational SMC ELBO=-1.77874283e+02 at epoch 5
07/12/2018 13:35:10 Variational SMC ELBO=-1.99352280e+02 at epoch 6
07/12/2018 13:35:11 Variational SMC ELBO=-1.73228989e+02 at epoch 7
07/12/2018 13:35:11 Variational SMC ELBO=-1.91512161e+02 at epoch 8
07/12/2018 13:35:11 Variational SMC ELBO=-1.91933380e+02 at epoch 9
07/12/2018 13:35:12 Variational SMC ELBO=-1.90392517e+02 at epoch 10
07/12/2018 13:35:12 Variational SMC ELBO=-1.82251404e+02 at epoch 11
07/12/2018 13:35:12 Variational SMC ELBO=-1.87149078e+02 at epoch 12
07/12/2018 13:35:13 Variational SMC ELBO=-1.79557755e+02 at epoch 13
07/12/2018 13:35:13 Variational SMC ELBO=-1.66635941e+02 at epoch 14
07/12/2018 13:35:13 Variational SMC ELBO=-1

In [15]:
for t in range(50):
    t += 1
    key = 'Z_%d' % t
    accuracy = (inference[key].value == generative[key].value).sum().to(dtype=torch.float) / inference.num_particles
    print('SMC accuracy at time %d: %f' % (t, accuracy))

SMC accuracy at time 1: 0.570000
SMC accuracy at time 2: 0.570000
SMC accuracy at time 3: 0.430000
SMC accuracy at time 4: 0.570000
SMC accuracy at time 5: 0.000000
SMC accuracy at time 6: 0.330000
SMC accuracy at time 7: 0.000000
SMC accuracy at time 8: 0.000000
SMC accuracy at time 9: 0.570000
SMC accuracy at time 10: 0.070000
SMC accuracy at time 11: 0.070000
SMC accuracy at time 12: 0.500000
SMC accuracy at time 13: 0.000000
SMC accuracy at time 14: 0.400000
SMC accuracy at time 15: 0.240000
SMC accuracy at time 16: 0.170000
SMC accuracy at time 17: 0.500000
SMC accuracy at time 18: 0.070000
SMC accuracy at time 19: 0.240000
SMC accuracy at time 20: 0.000000
SMC accuracy at time 21: 1.000000
SMC accuracy at time 22: 0.000000
SMC accuracy at time 23: 0.300000
SMC accuracy at time 24: 0.440000
SMC accuracy at time 25: 0.760000
SMC accuracy at time 26: 0.200000
SMC accuracy at time 27: 0.000000
SMC accuracy at time 28: 0.000000
SMC accuracy at time 29: 0.170000
SMC accuracy at time 30

In [16]:
smc_hmm_params

"{'Pi_0__concentration': 'Parameter containing:\ntensor([ 1.4492,  1.3633,  1.3943,  1.3601,  1.5518])', 'Pi_1__concentration': 'Parameter containing:\ntensor([ 0.8448,  1.2033,  1.0076,  0.8605,  1.6219])', 'Pi_2__concentration': 'Parameter containing:\ntensor([ 0.9631,  0.8069,  0.9238,  0.9986,  1.4788])', 'Pi_3__concentration': 'Parameter containing:\ntensor([ 1.1822,  0.9690,  0.7210,  0.9652,  1.5765])', 'Pi_4__concentration': 'Parameter containing:\ntensor([ 1.0780,  1.0275,  1.0581,  0.7093,  1.5689])', 'Pi_5__concentration': 'Parameter containing:\ntensor([ 1.0148,  1.0273,  0.8070,  1.0160,  1.7135])', 'mu__loc': 'Parameter containing:\ntensor([ 2.5311,  2.5526,  2.2992,  2.4523,  3.0499])', 'mu__scale': 'Parameter containing:\ntensor([ 2.3983,  2.4903,  2.3207,  2.5532,  2.7303])', 'sigma__loc': 'Parameter containing:\ntensor([ 2.6862,  2.5644,  2.5864,  2.7013,  2.9132])', 'sigma__scale': 'Parameter containing:\ntensor([ 1.0580,  1.1261,  1.1160,  1.1246,  0.4327])'}"